In [1]:
# https://github.com/helgefmi/ohno/blob/master/ohno/client/pty.py ## loosely based on this code
import os
import pty
import subprocess
import pprint
import time
import json
import select
import stransi
import re

# I'm changing these from raw strings to the actual escape sequences
ansi_escape_codes = {
    "\x1b7": "save cursor",
    "\x1b\[\?47h": "set alternate screen",
    "\x1b\)0": "set to ASCII",
    "\x1b\[H": "move cursor to upper left",
    "\x1b\[2J": "clear screen", # this might be a parameter?
    "\x1b\[(\d+);(\d+)H": "move cursor",
    "\x1b\[K": "clear to end of line",
    "\x1b\[C": "move cursor right",
    "\x1b8": "restore cursor",
}

class Terminal:
    ## this dictionary *is* used, which could be an issue.
    # let's take out the r"" prefix and see what happens
    unsupported = {
        "\x1b7": "save cursor",
        "\x1b8": "restore cursor",
        "\x1b\[\?47h": "set alternate screen",
        "\x1b\)0": "set to ASCII",
    }

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.reset()

    def reset(self):
        self.cursor_row = 0
        self.cursor_col = 0
        self.characters = [[' ' for i in range(self.cols)] for _ in range(self.rows)]
        self.foregrounds = [["white" for i in range(self.cols)] for _ in range(self.rows)]
        self.backgrounds = [["black" for i in range(self.cols)] for _ in range(self.rows)] # does Nethack actually do this?

    def text(self):
        s = ""
        for i in range(self.rows):
            for j in range(self.cols): # let's ignore colors for now
                s += self.characters[i][j]
            s += "\n"
        return s
    
    def print(self):
        print(self.text())
                
    def update(self, input, print_instructions = False):
        #print_instructions = True # for now
        #self.reset()
        # this is where the logic is going to get complex
        cleaned = input
        for k, v in Terminal.unsupported.items():
            # first we need to check if the key is in the input
            if re.search(k, cleaned):
                print(f"found unsupported {k} : {v}")
            rgx = re.compile(k)
            cleaned = re.sub(rgx, "", cleaned)
        ansi = stransi.Ansi(cleaned)
        for instr in ansi.instructions():
            if print_instructions: 
                print(instr) # for now
            if isinstance(instr, str):
                for i in range(len(instr)):
                    ch = instr[i]
                    if ch == "\r": # carriage return; reset cursor column to 0
                        self.cursor_col = 0
                    elif ch == "\b": # backspace; move cursor column back by one
                        self.cursor_col -= 1
                    elif ch == "\n": # newline; move cursor row down by one
                        self.cursor_row += 1
                    elif ch == '\x0f' or ch == '\x00': # these should indeed be no-ops but I'm not sure why they're here
                        pass
                    else:
                        try:
                            self.characters[self.cursor_row][self.cursor_col] = ch
                            self.cursor_col += 1
                        except:
                            print(f"something out of bounds: col={self.cursor_col}, row={self.cursor_row}, i={i} len={len(instr)}")
            elif isinstance(instr, stransi.SetCursor):
                if instr.move.relative: # Move the cursor relative to its current position
                    self.cursor_row += instr.move.x
                    #self.cursor_col += instr.move.y
                    self.cursor_col -= instr.move.y # let's give this a try I guess?
                else: # Move the cursor to an absolute position
                    self.cursor_row = instr.move.x
                    self.cursor_col = instr.move.y 
            elif isinstance(instr, stransi.SetClear):
                if instr.region == stransi.clear.Clear.SCREEN:
                    self.reset()
                elif instr.region == stransi.clear.Clear.LINE_AFTER:
                    for i in range(self.cursor_col, self.cols):
                        self.characters[self.cursor_row][i] = ' ' # clear the rest of the line
                elif instr.region == stransi.clear.Clear.LINE:
                    print("stransi.clear.Clear.LINE, was not expecting this.")
                    for i in range(self.cols):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.LINE_BEFORE:
                    print("stransi.clear.Clear.LINE_BEFORE, was not expecting this.")
                    for i in range(0, self.cursor_col):
                        self.characters[self.cursor_row][i] = ' '
                elif instr.region == stransi.clear.Clear.SCREEN_AFTER:
                    print("stransi.clear.Clear.SCREEN_AFTER, was not expecting this.")
                    for i in range(self.cursor_row, self.rows):
                        for j in range(self.cols):
                            self.characters[i][j] = ' '
                elif instr.region == stransi.clear.Clear.SCREEN_BEFORE:
                    print("stransi.clear.Clear.SCREEN_BEFORE, was not expecting this.")
                    for i in range(0, self.cursor_row):
                        for j in range(self.cols):
                            self.characters[i][j] = ' '
                else:
                    print(instr)
                    print(instr.region)
                    print("I guess I didn't handle all the clears")
            elif isinstance(instr, stransi.SetColor):
                if instr.role == stransi.color.ColorRole.FOREGROUND:
                    self.foregrounds[self.cursor_row][self.cursor_col] = instr.color.ansi256.code
                elif instr.role == stransi.color.ColorRole.BACKGROUND:
                    self.backgrounds[self.cursor_row][self.cursor_col] = instr.color.ansi256.code
                else:
                    print("some other kind of color instruction?")
                    print(instr)
            elif isinstance(instr, stransi.SetAttribute):
                if instr.attribute == stransi.attribute.Attribute.NORMAL:
                    pass
                    #print("normal")
                elif instr.attribute == stransi.attribute.Attribute.BOLD:
                    pass
                    #print("bold")
                elif instr.attribute == stransi.attribute.Attribute.REVERSE:
                    pass
                    #print("reverse")
                else:
                    print("another attribute?")
                    print(instr)
            else:
                #if print_instructions:
                print("^^^ this instruction was not handled.")
        # this is how the version I'm drawing from did it
        #if self.cursor_col >= self.cols:
        while self.cursor_col >= self.cols:
            self.cursor_col = self.cursor_col - self.cols # wrap around to the next line
            self.cursor_row += 1
        #if self.cursor_row >= self.rows:
        while self.cursor_row >= self.rows:
            self.cursor_row = self.rows - 1 # don't go off the screen


class Nethack:
    def __init__(self):
        self.master = None
        self.slave = None
        self.process = None
        self.terminal = Terminal(24, 80)

    def start_process(self):
        self.master, self.slave = pty.openpty()
        self.process = subprocess.Popen(
            ["nethack"],
            stdin=self.slave,
            stdout=self.slave,
            stderr=self.slave,
            close_fds=True, # this line comes from Bing Chat and I don't know what it does really
            universal_newlines=True,
        )
        os.close(self.slave) # Copilot added this so take it with a grain of salt...if it works, we don't need to save the reference

    def read_output(self, timeout = 1.0, sleep = 0.25, chars = 8192):
        if self.process is None:
            raise ValueError("Process not started")
        if sleep is not None and sleep > 0:
            time.sleep(sleep)
        if timeout is not None and timeout > 0:
            rlist, _, _ = select.select([self.master], [], [], timeout)
            if rlist:
                return os.read(self.master, chars).decode()
            else:
                print("(read timed out)")
                print("(this usually means the input sent did not update the game state.)")
                return None
        else:
            return os.read(self.master, chars).decode()

    def send_command(self, command):
        #os.write(self.master, command.encode() + b"\n")
        os.write(self.master, command.encode())

    def close(self):
        try:
            self.process.terminate()
            os.close(self.master)
        except:
            pass

    def next_command(self, c, verbose = False, timeout = 1.0):
        self.send_command(c)
        text = self.read_output(timeout = timeout)
        if verbose:
            print(text)
        if text is not None:
            self.terminal.update(text, print_instructions = verbose)
        print(self.terminal.text())
        return text

    @staticmethod
    def clean_locks():
        password = json.load(open("private.json"))["sudo"]
        os.system(f'echo "{password}" | sudo -S rm -f /var/games/nethack/*lock*')

    @staticmethod
    def get_new_process(verbose = False):
        Nethack.clean_locks()
        n = Nethack()
        n.start_process()
        text = n.read_output(timeout = 1.0)
        n.terminal.update(text, print_instructions = verbose)
        print(n.terminal.text())
        return n

# 24, 80

In [2]:
ENTER = "\x0d"

In [3]:
nethack = Nethack.get_new_process()

[sudo] password for perplexity: 

found unsupported 7 : save cursor
found unsupported \[\?47h : set alternate screen
found unsupported \)0 : set to ASCII
                                                                                
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
Shall I pick character's race, role, gender and alignment for you? [ynaq]       
                                                                                
                                                                                
                                                                   

In [4]:
nethack.next_command("n")

       name: perplexity                  Pick a role or profession              
       role:  choosing now                                                      
       race:  not yet specified          a - an Archeologist                    
     gender:  not yet specified          b - a Barbarian                        
  alignment:  not yet specified          c - a Caveman/Cavewoman                
                                         h - a Healer                           
                                         k - a Knight                           
                                         m - a Monk                             
                                         p - a Priest/Priestess                 
                                         r - a Rogue                            
                                         R - a Ranger                           
                                         s - a Samurai                          
                            

'n\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role:  choosing now\r\x1b[3;1H       race:  not yet specified\r\x1b[4;1H     gender:  not yet specified\r\x1b[5;1H  alignment:  not yet specified\x1b[1;41H\x1b[K Pick a role or profession\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K a - an Archeologist\x1b[4;41H\x1b[K b - a Barbarian\x1b[5;41H\x1b[K c - a Caveman/Cavewoman\x1b[6;41H\x1b[K h - a Healer\x1b[7;41H\x1b[K k - a Knight\x1b[8;41H\x1b[K m - a Monk\x1b[9;41H\x1b[K p - a Priest/Priestess\x1b[10;41H\x1b[K r - a Rogue\x1b[11;41H\x1b[K R - a Ranger\x1b[12;41H\x1b[K s - a Samurai\x1b[13;41H\x1b[K t - a Tourist\x1b[14;41H\x1b[K v - a Valkyrie\x1b[15;41H\x1b[K w - a Wizard\x1b[16;41H\x1b[K * + Random\x1b[17;41H\x1b[K \x1b[18;41H\x1b[K / - Pick race first\x1b[19;41H\x1b[K " - Pick gender first\x1b[20;41H\x1b[K [ - Pick alignment first\x1b[21;41H\x1b[K q - Quit\x1b[22;41H\x1b[K\x1b[C(end) '

In [5]:
nethack.next_command("v")

       name: perplexity                  Pick a race or species                 
       role: Valkyrie                                                           
       race:  choosing now               h - human                              
     gender: female                      d - dwarf                              
  alignment:  not yet specified          * + Random                             
                                                                                
                                         ? - Pick another role first            
                                             role forces female                 
                                         [ - Pick alignment first               
                                         q - Quit                               
                                                                                
                                        (end)                                   
                            

'\x1b[14;44H+\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[10;40H\x1b[K\x1b[11;40H\x1b[K\x1b[12;40H\x1b[K\x1b[13;40H\x1b[K\x1b[14;40H\x1b[K\x1b[15;40H\x1b[K\x1b[16;40H\x1b[K\x1b[17;40H\x1b[K\x1b[18;40H\x1b[K\x1b[19;40H\x1b[K\x1b[20;40H\x1b[K\x1b[21;40H\x1b[K\x1b[22;40H\x1b[K\x1b[23;40H\x1b[K\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role: Valkyrie\r\x1b[3;1H       race:  choosing now\r\x1b[4;1H     gender: female\r\x1b[5;1H  alignment:  not yet specified\x1b[1;41H\x1b[K Pick a race or species\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K h - human\x1b[4;41H\x1b[K d - dwarf\x1b[5;41H\x1b[K * + Random\x1b[6;41H\x1b[K \x1b[7;41H\x1b[K ? - Pick another role first\x1b[8;41H\x1b[K     role forces female\x1b[9;41H\x1b[K [ - Pick alignment first\x1b[10;41H\x1b[K q - Quit\x1b[11;41H\x1b[K\x1b[C(end) '

In [6]:
nethack.next_command("h")

       name: perplexity                  Pick an alignment or creed             
       role: Valkyrie                                                           
       race: human                       l - lawful                             
     gender: female                      n - neutral                            
  alignment:  choosing now               * + Random                             
                                                                                
                                         ? - Pick another role first            
                                         / - Pick another race first            
                                             role forces female                 
                                         q - Quit                               
                                                                                
                                        (end)                                   
                            

'\x1b[3;44H+\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[10;40H\x1b[K\x1b[11;40H\x1b[K\x1b[12;40H\x1b[K\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role: Valkyrie\r\x1b[3;1H       race: human\r\x1b[4;1H     gender: female\r\x1b[5;1H  alignment:  choosing now\x1b[1;41H\x1b[K Pick an alignment or creed\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K l - lawful\x1b[4;41H\x1b[K n - neutral\x1b[5;41H\x1b[K * + Random\x1b[6;41H\x1b[K \x1b[7;41H\x1b[K ? - Pick another role first\x1b[8;41H\x1b[K / - Pick another race first\x1b[9;41H\x1b[K     role forces female\x1b[10;41H\x1b[K q - Quit\x1b[11;41H\x1b[K\x1b[C(end) '

In [7]:
nethack.next_command("n")

       name: perplexity                  Is this ok? [ynq]                      
       role: Valkyrie                                                           
       race: human                       perplexity, neutral human Valkyrie     
     gender: female                                                             
  alignment: neutral                     y + Yes; start game                    
                                         n - No; choose role again              
                                         q - Quit                               
                                                                                
                                        (end)                                   
                                                                                
                                                                                
                                                                                
                            

'\x1b[4;44H+\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[10;40H\x1b[K\x1b[11;40H\x1b[K\x1b[12;40H\x1b[K\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role: Valkyrie\r\x1b[3;1H       race: human\r\x1b[4;1H     gender: female\r\x1b[5;1H  alignment: neutral\x1b[1;41H\x1b[K Is this ok? [ynq]\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K perplexity, neutral human Valkyrie\x1b[4;41H\x1b[K \x1b[5;41H\x1b[K y + Yes; start game\x1b[6;41H\x1b[K n - No; choose role again\x1b[7;41H\x1b[K q - Quit\x1b[8;41H\x1b[K\x1b[C(end) '

In [8]:
nethack.next_command("y")

                       It is written in the Book of Odin:                       
                                                                                
                           After the Creation, the cruel god Moloch rebelled    
                           against the authority of Marduk the Creator.         
                           Moloch stole from Marduk the most powerful of all    
                           the artifacts of the gods, the Amulet of Yendor,     
                           and he hid it in the dark cavities of Gehennom, the  
                           Under World, where he now lurks, and bides his time. 
                                                                                
                       Your god Odin seeks to possess the Amulet, and with it   
                       to gain deserved ascendance over the other gods.         
                                                                                
                       You, 

'\x1b[5;44H-\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[H\x1b[2J\x1b[H\x1b[H\x1b[2J\x1b[H\x1b[17;22H-.--.----------\x1b[m\x1b[18;22H|.............|\x1b[m\x1b[19;22H|....\x1b[m\x1b\x1b$\x1b[m........|\x1b[m\x1b[20;22H|....\x1b[m\x1b\x1b@\x1bd\x1b[m\x1b[m.......|\x1b[m\x1b[21;22H---------------\x1b[m\x1b[20;27H\r\x1b[23;1H\x1b[K\x1b[24;1H\x1b[K\x1b[A[\x1bPerplexity the Stripling      \x1b[m] St:18 Dx:12 Co:18 In:10 Wi:8 Ch:9 Neutral\x1b[K\r\x1b[24;1HDlvl:1 $:0 HP:16(16) Pw:2(2) AC:6 Xp:1\x1b[K\x1b[1;23H\x1b[K It is written in the Book of Odin:\x1b[2;23H\x1b[K \x1b[3;23H\x1b[K     After the Creation, the cruel god Moloch rebelled\x1b[4;23H\x1b[K     against the authority of Marduk the Creator.\x1b[5;23H\x1b[K     Moloch stole from Marduk the most powerful of all\x1b[6;23H\x1b[K     the artifacts of the gods, the Amulet of Yendor,\x1b[7;23H\x1b[K     and he hid it in the dark cavities o

In [10]:
RETURN = "\x0d"
LEFT = "h"
RIGHT = "l"
UP = "k"
DOWN = "j"
nethack.next_command(RETURN)

Be careful!  New moon tonight.                                                  
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                            

'\r\x1b[A\x1b[K\x1b[2;1H\x1b[K\x1b[H\x1b[HBe careful!  New moon tonight.\x1b[K\x1b[23;1H[\x1bPerplexity the Stripling      \x1b[m] St:18 Dx:12 Co:18 In:10 Wi:8 Ch:9 Neutral\r\x1b[24;1HDlvl:1 $:0 HP:16(16) Pw:2(2) AC:6 Xp:1\x1b[20;27H'

In [13]:
nethack.next_command(UP, timeout = 0.5)

You swap places with Slinky.  $ - 7 gold pieces.                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                            

'\x1b[A\x1b\x1b@\x1b[m\x1b[20;27H\x1b\x1b\x1bd\x1b[m\x1b[m\x1b[A\x08\x1b[HYou swap places with Slinky.\x1b[K\x1b[19;27H\x1b[23;1H\x1b[24;1H\x1b[24;7H $:7\x1b[1;31H$ - 7 gold pieces.\x1b[K\x1b[18;26H\x1b\x1b\x1bd\x1b[m\x1b[m\x1b[19;26H.\x1b[m\x1b\x1b@\x1b[m\x1b[20;27H<\x1b[m\x1b[A\x08'

In [16]:
nethack.next_command("h", timeout = 0.5)

                                                                                
                                                                                
                                  -------                                       
                                  |..@f.|                                       
                                  +.f..@|                                       
                                  |.....|                                       
                                  -----.                                        
                                                                                
                                                                                
                                                                                
                                         .                                      
                                         x.                                     
                            

'\x08\x1b\x1b@\x1bf\x1b[m\x1b[m\x1b[5;39H.\x1b[m\x1b[6;40H.\x1b[m\x1b[12;42H\x1bx\x1b[m.\x1b[m\x1b[13;43H.\x1b[m\x1b[14;43H..\x1b[m\x1b[15;44H..\x1b[m\x1b[4;38H'

In [15]:
# h is left, j is down, k is up, l is right
#text = next_command("j")  # okay, something odd is happening here: When I move, the player cursor disappears, and then the right way moves to the left.  So it seems like some portion of the text is getting blotted out.
# interesting, that final character is \x08 which is backspace.  I wonder if that has smething to do with it.
nethack.next_command("k", timeout = 0.5, verbose = True)
# The problems I'm seeing here seem to be of a form where the I see the new locations of entities, but the old locations are not being cleared.
# It's actually a bit weirder than that.  There's some jumping around.  That kind of makes it seem like some of the control characters are not being handled correctly.
# Which sucks, because I know I already put a ton of work into that.  And I know it's a pain to debug.
# So, one question here is whether the things are not updating, or whether they're updating incorrectly.  That should actually not be too hard to figure out; we could just 
# clear the screen either to blanks or to a single character, and then see if the entities are being drawn correctly.
# So it looks to be a matter of not updating things that should be updating; however, we're also getting some strange things...
# ...the player did not get drawn after switching places with the pet.
# ...sometimes the player "teleports" to the top-center of the screen.
# When I get rid of the full clear, we do see some teleporting around.  You might expect that if the screen is getting re-centered, but then you would expect other things to happen as well.

d@...d[
SetCursor(move=CursorMove(x=0, y=-1, relative=True))

SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(3))
d
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=1, y=0, relative=True))
SetAttribute(attribute=<Attribute.BOLD: 1>)
SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(7))
@
SetAttribute(attribute=<Attribute.NORMAL: 0>)
.
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=16, y=65, relative=False))
.
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=1, y=0, relative=True))
SetCursor(move=CursorMove(x=1, y=0, relative=True))
.
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=17, y=68, relative=False))
SetAttribute(attribute=<Attribute.BOLD: 1>)
SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(7))
SetAttribute(attribute=<Attribute.REVERSE: 7>)
d
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=Cursor

'\x1b[A\x08\x08\x1bd\x1b[m\x1b[C\x1b\x1b@\x1b[m.\x1b[m\x1b[17;66H.\x1b[m\x1b[C\x1b[C.\x1b[m\x1b[18;69H\x1b\x1b\x1bd\x1b[m\x1b[m\x1b[A\x1b[A\x08'